In [1]:
from google import genai
from google.genai import types
import pandas as pd
import json
from itertools import cycle
import ctypes
import time
from random import randint
from loguru import logger 
from tenacity import retry, stop_after_delay, stop_after_attempt
import os

client = genai.Client(api_key='GEMINI_API_KEY')

In [2]:
@retry(stop=(stop_after_delay(1) | stop_after_attempt(3)))
def generate_content(prompt, system_prompt = "", temperature=0.1, max_output_tokens=4096):
    safety_settings = [
                                                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                                                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
                                                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                                                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                                            ]
    if system_prompt:
        config = types.GenerateContentConfig(max_output_tokens=max_output_tokens,
                                            temperature=temperature,
                                            system_instruction=[
                                                system_prompt
                                            ],
                                            safety_settings=safety_settings
                                            )
    else:
        config = types.GenerateContentConfig(max_output_tokens=max_output_tokens,
                                            temperature=temperature,
                                            safety_settings=safety_settings
                                            )
    result = client.models.generate_content(model="models/gemini-2.5-flash-preview-05-20",
                                            contents=[
                                                prompt
                                            ], 
                                            config = config
    )
    result_text = result.text
    if not result_text:
        raise Exception()
    return result_text

In [3]:
def decompose_problem(problem):
    """Prompt the model to decompose the problem into steps."""
    prompt = f"""
    You are an expert problem solver. Given the following problem, break it down into clear, logical steps needed to solve it. Provide only the steps as a numbered list, no explanations:
    
    Problem: {problem}
    
    Steps:
    """
    system_prompt = "You are a precise and logical problem solver."
    response_text = generate_content(prompt, system_prompt = system_prompt, temperature=0.2, max_output_tokens=4096)
    logger.info(response_text)
    steps = response_text.strip().split("\n")[2:]  # Skip "Steps:" header
    return [step.strip() for step in steps if step.strip()]

def execute_step(step, context):
    """Execute a single reasoning step using the model, with context from previous steps."""
    prompt = f"""
    Given the problem and previous context, solve the following step and provide the result. If calculations are needed, show them clearly. If the step involves reasoning, explain briefly. Return only the result and a brief explanation (if needed).

    Problem: {context['problem']}
    Previous context: {context.get('previous_steps', [])}
    Current step: {step}
    
    Result:
    """
    system_prompt = "You are a precise and logical problem solver."
    response_text = generate_content(prompt, system_prompt = system_prompt, temperature=0.2, max_output_tokens=4096)
    logger.info(response_text)
    return response_text.strip().split("\n")[-1]  # Extract result

def validate_step(step, result, context):
    """Validate the result of a step and decide if refinement is needed."""
    prompt = f"""
    You are a validator for a reasoning step. Given the problem, the step, and its result, check if the result is correct and logical. If incorrect or unclear, suggest a correction or clarification. Return 'Valid' if correct, or a correction suggestion if not.

    Problem: {context['problem']}
    Step: {step}
    Result: {result}
    
    Validation:
    """
    system_prompt = "You are a precise validator of logical steps."
    response_text = generate_content(prompt, system_prompt = system_prompt, temperature=0.2, max_output_tokens=4096)
    logger.info(response_text)
    return response_text.strip().split("\n")[-1]

def autodicot(problem):
    """Main AutoDiCoT function to solve a problem with directed reasoning."""
    context = {"problem": problem, "previous_steps": []}
    
    # Step 1: Decompose the problem
    steps = decompose_problem(problem)
    print("Decomposed Steps:", steps)
    
    results = []
    for step in steps:
        # Step 2: Execute the step
        result = execute_step(step, context)
        print(f"Step: {step}\nResult: {result}")
        
        # Step 3: Validate the result
        validation = validate_step(step, result, context)
        if validation != "Valid":
            print(f"Validation failed: {validation}")
            # Refine by re-executing with correction suggestion
            context["previous_steps"].append(f"{step}: {result} (Correction: {validation})")
            result = execute_step(step, context)  # Re-run with updated context
            print(f"Refined Result: {result}")
        
        context["previous_steps"].append(f"{step}: {result}")
        results.append(result)
        time.sleep(1)
        
    # Step 4: Consolidate final answer
    final_prompt = f"""
    Given the problem and all reasoning steps, provide the final answer concisely.
    
    Problem: {problem}
    Reasoning steps: {context['previous_steps']}
    
    Final Answer:
    """
    system_prompt = "You are a precise problem solver."
    response_text = generate_content(final_prompt, system_prompt = system_prompt, temperature=0.2, max_output_tokens=4096)
    logger.info(response_text)
    final_answer = response_text.strip().split("\n")[-1]
    return final_answer


In [4]:
problem = "A store offers a 20% discount on a $50 item. If there's a 5% sales tax applied after the discount, what is the final price?"
final_answer = autodicot(problem)
print(f"\nFinal Answer: {final_answer}")

2025-06-16 10:19:02.094 | INFO     | __main__:decompose_problem:12 - 1. Calculate the discount amount.
2. Subtract the discount amount from the original price to find the price after discount.
3. Calculate the sales tax amount on the discounted price.
4. Add the sales tax amount to the discounted price to find the final price.


Decomposed Steps: ['3. Calculate the sales tax amount on the discounted price.', '4. Add the sales tax amount to the discounted price to find the final price.']


2025-06-16 10:19:03.691 | INFO     | __main__:execute_step:29 - Sales tax amount = 5% of $40 = 0.05 * 40 = $2.00


Step: 3. Calculate the sales tax amount on the discounted price.
Result: Sales tax amount = 5% of $40 = 0.05 * 40 = $2.00


2025-06-16 10:19:05.689 | INFO     | __main__:validate_step:45 - Valid
2025-06-16 10:19:07.655 | INFO     | __main__:execute_step:29 - Final price = $40 + $2.00 = $42.00


Step: 4. Add the sales tax amount to the discounted price to find the final price.
Result: Final price = $40 + $2.00 = $42.00


2025-06-16 10:19:10.663 | INFO     | __main__:validate_step:45 - Valid
2025-06-16 10:19:12.328 | INFO     | __main__:autodicot:86 - The final price is $42.00.



Final Answer: The final price is $42.00.


In [5]:
problem = """A researcher is studying the effectiveness of two teaching methods (A and B) on student exam scores.
A sample of 50 students is randomly split into two groups: 25 students are taught using Method A, and 25 using Method B.
The exam scores (out of 100) for Method A have a mean of 78 and a standard deviation of 10, while Method B scores have a mean of 82 and a standard deviation of 8.
Assume the scores are normally distributed.
The researcher wants to determine if there is a statistically significant difference between the two methods at a 5% significance level using a two-sample t-test.
Additionally, calculate the 95% confidence interval for the difference in mean scores.
What is the conclusion, and what is the confidence interval?"""
final_answer = autodicot(problem)
print(f"\nFinal Answer: {final_answer}")

2025-06-16 10:19:19.300 | INFO     | __main__:decompose_problem:12 - 1.  State the null and alternative hypotheses.
2.  Identify the significance level (alpha).
3.  Determine the appropriate formula for the two-sample t-statistic, considering whether to assume equal or unequal variances.
4.  Calculate the t-statistic.
5.  Calculate the degrees of freedom for the t-test.
6.  Determine the critical t-value(s) for the given significance level and degrees of freedom, or calculate the p-value.
7.  Compare the calculated t-statistic to the critical t-value(s) or compare the p-value to the significance level.
8.  Make a decision regarding the null hypothesis (reject or fail to reject).
9.  State the conclusion of the hypothesis test in the context of the problem.
10. Calculate the standard error of the difference between the means.
11. Determine the t-multiplier for the 95% confidence interval using the calculated degrees of freedom.
12. Calculate the 95% confidence interval for the differenc

Decomposed Steps: ['3.  Determine the appropriate formula for the two-sample t-statistic, considering whether to assume equal or unequal variances.', '4.  Calculate the t-statistic.', '5.  Calculate the degrees of freedom for the t-test.', '6.  Determine the critical t-value(s) for the given significance level and degrees of freedom, or calculate the p-value.', '7.  Compare the calculated t-statistic to the critical t-value(s) or compare the p-value to the significance level.', '8.  Make a decision regarding the null hypothesis (reject or fail to reject).', '9.  State the conclusion of the hypothesis test in the context of the problem.', '10. Calculate the standard error of the difference between the means.', '11. Determine the t-multiplier for the 95% confidence interval using the calculated degrees of freedom.', '12. Calculate the 95% confidence interval for the difference in mean scores.', '13. State the final conclusion based on both the hypothesis test and the confidence interval.

2025-06-16 10:19:24.495 | INFO     | __main__:execute_step:29 - The appropriate formula for the two-sample t-statistic, assuming unequal variances (Welch's t-test), is:

$t = \frac{(\bar{x}_1 - \bar{x}_2) - (\mu_1 - \mu_2)}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}$

This formula is chosen because the standard deviations of the two samples are different (10 and 8), and there is no information or instruction to assume equal variances. The Welch's t-test is more robust when the assumption of equal variances might not hold.


Step: 3.  Determine the appropriate formula for the two-sample t-statistic, considering whether to assume equal or unequal variances.
Result: This formula is chosen because the standard deviations of the two samples are different (10 and 8), and there is no information or instruction to assume equal variances. The Welch's t-test is more robust when the assumption of equal variances might not hold.


2025-06-16 10:19:26.160 | INFO     | __main__:validate_step:45 - Valid
2025-06-16 10:19:29.846 | INFO     | __main__:execute_step:29 - t = (x̄1 - x̄2) / sqrt((s1^2 / n1) + (s2^2 / n2))
t = (78 - 82) / sqrt((10^2 / 25) + (8^2 / 25))
t = -4 / sqrt((100 / 25) + (64 / 25))
t = -4 / sqrt(4 + 2.56)
t = -4 / sqrt(6.56)
t = -4 / 2.5612496
t ≈ -1.5617


...

Refined Result: CI = [-9.158, 1.158]


2025-06-16 10:21:10.060 | INFO     | __main__:execute_step:29 - Based on the hypothesis test, we fail to reject the null hypothesis, indicating no statistically significant difference between the two teaching methods at the 5% significance level. The 95% confidence interval for the difference in mean scores is [-9.158, 1.158]. Since this interval includes zero, it supports the conclusion that there is no statistically significant difference between the mean scores of Method A and Method B.


Step: 13. State the final conclusion based on both the hypothesis test and the confidence interval.
Result: Based on the hypothesis test, we fail to reject the null hypothesis, indicating no statistically significant difference between the two teaching methods at the 5% significance level. The 95% confidence interval for the difference in mean scores is [-9.158, 1.158]. Since this interval includes zero, it supports the conclusion that there is no statistically significant difference between the mean scores of Method A and Method B.


2025-06-16 10:21:18.712 | INFO     | __main__:validate_step:45 - Valid
2025-06-16 10:21:20.656 | INFO     | __main__:autodicot:86 - The conclusion is that there is no statistically significant difference between the two teaching methods (A and B) at the 5% significance level. The 95% confidence interval for the difference in mean scores (Method A - Method B) is [-9.158, 1.158].



Final Answer: The conclusion is that there is no statistically significant difference between the two teaching methods (A and B) at the 5% significance level. The 95% confidence interval for the difference in mean scores (Method A - Method B) is [-9.158, 1.158].
